In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='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'
)

In [65]:
from datetime import datetime

# Get current date
c_date = datetime.now()

# Format the date as 'Month Day, Year'
current_date = c_date.strftime("%B %d, %Y")

print(current_date)


September 17, 2024


In [2]:
response = llm.invoke("who invented usa")
print(response.content)

The United States of America (USA) was not invented by a single person. Instead, it was formed through a series of events, agreements, and the contributions of many individuals. Here's a brief overview:

**Indigenous Peoples**: The first inhabitants of the land that is now the United States were Native American tribes, who had been living there for thousands of years.

**European Exploration**: In the late 15th century, European explorers, such as Christopher Columbus (Italy) and Vasco Nunez de Balboa (Spain), began to visit and map the eastern coast of North America.

**British Colonization**: In the early 17th century, the British established the first successful colonies in what is now the United States, including Jamestown (Virginia) in 1607 and Plymouth Colony (Massachusetts) in 1620.

**Founding Fathers**: A group of visionary individuals, known as the Founding Fathers, played a crucial role in shaping the United States. Some of the most notable Founding Fathers include:

1. **Ge

In [77]:
# WebBaseLoader to load all text from HTML webpages into a document format that we can use downstream
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.takeda.com/job/-/-/1113/68535121024?source=tmp_takeda_linkedin_jobads&utm_source=linkedin.com&utm_medium=job_posting&utm_campaign=Enterprise&utm_content=social_media&utm_term=283784093&ss=paid&dclid=CMP8-6SayogDFa2tgwgdw6cIhA")

page_data = loader.load().pop().page_content
print(page_data)

















Cell Therapy Data Engineer at Takeda Pharmaceutical









































































































Skip to main content






Careers





Jobs

Culture

Living Our Values
Diversity, Equity & Inclusion
Employee Stories



Teams

Career Areas
Business Units

Corporate Functions
Europe & Canada
Global Manufacturing & Supply
Global Quality
Japan Pharma
Oncology
Plasma-derived Therapies
RESEARCH & DEVELOPMENT
US Commercial Organization
Vaccines





Locations

Global Locations

Greater Boston Area
Zurich
Tokyo





Biolife



Learn more about our BioLife plasma donation centers.
BioLife Careers


Saved Jobs
Join Our Talent Community
TAKEDA.COM
















Search Jobs


Find jobs for



Location




Radius
Select miles
5 miles
15 miles
25 miles
35 miles
50 miles



Search Jobs








Cell Therapy Data Engineer
Boston, Massachusetts


Apply Now
Save







                        Jump To
                        



In [78]:
# creating prompt to extract data i.e. roles, experience, skills, etc from the scrapped data
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: 'company_name', 'company_address', 'role', 'experience', 'skills', 'description' and 'about'.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

# using pipe operation - to create a chain from getting a prompt and passing it to llm
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [84]:
# converting str Response into JSON Response
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res = json_res[0]

In [85]:
type(json_res)

dict

In [86]:
json_res['description']

'Design and carry out experiments, Manage multiple projects, Document data, Mentor other scientists'

In [87]:
prompt_cover_letter = PromptTemplate.from_template(
    """
    SHRUTI MILIND RANDIVE  
    (+1) 224-522-8056 | Boston, MA | randive.s@northeastern.edu

    {current_date}

    Hiring Manager  
    Human Relations Department  
    {company_name}  
    {company_address}

    Dear Hiring Manager,

    I am excited to apply for the {role} position at {company_name}. With a Master of Science in Information Systems from Northeastern University and a background in Computer Engineering, I have developed a robust foundation in data engineering and analytics. My two years of experience as a Python developer have sharpened my skills in designing, developing, and deploying scalable data solutions.

    In my previous roles, I have successfully developed a Flask application for ML-based prostate cancer diagnosis as a Graduate Research Assistant at Northeastern University, enhancing decision-making precision by 20%. At Kiddo.Health, my work as a Data Science Intern involved designing and implementing Streamlit dashboards, and optimizing data pipelines using AWS Glue, S3, and Delta Lake, which significantly improved time series data processing efficiency. Additionally, during my tenure as a Software Developer at Apexon (Infostretch), I engineered REST APIs, migrated data to AWS, and built serverless applications, achieving a 50% improvement in both security and scalability.

    I am impressed by {company_name}'s commitment to {company_info}. With my expertise in Python, SQL, and cloud services (AWS and Azure), coupled with my experience in data engineering tools like Talend and Snowflake, I am confident in my ability to make a meaningful contribution to your team. My proficiency in data modeling, schema design, and data tools/ops, along with advanced skills in SQL, dbt, Python, cloud data warehouses, and data infra/ops tools, aligns well with the role's requirements. Additionally, my experience in data visualization with Power BI and Tableau will support your goals of delivering insightful and actionable data solutions.

    I am enthusiastic about the opportunity to contribute to {company_name}'s innovative goals and would welcome the chance to discuss how my skills align with your team's needs. I look forward to the possibility of discussing my experience and qualifications further for the data engineering position at {company_name}. Please feel free to contact me at 224-522-8056 or randive.s@northeastern.edu. Thank you very much for your time and consideration.

    Please find my resume attached for your review.

    Sincerely,  
    Shruti Milind Randive
    Do not provide a preamble.
    """
)


In [88]:
chain_cover_letter = prompt_cover_letter | llm

# Example of how to invoke it without links
res = chain_cover_letter.invoke({
    "company_name": str(json_res['company_name']),
    "job_description": str(json_res['description']), 
    "company_address": str(json_res['company_address']), 
    "company_info": str(json_res['about']),
    "role": str(json_res['role']), 
    "skills": str(json_res['skills']),    
    "experience": str(json_res['experience']),    
    "current_date": str(current_date)
})

# Printing the generated cover letter
print(res.content)

SHRUTI MILIND RANDIVE  
(+1) 224-522-8056 | Boston, MA | randive.s@northeastern.edu

September 17, 2024

Hiring Manager  
Human Relations Department  
Takeda Pharmaceutical  
Boston, Massachusetts

Dear Hiring Manager,

I am excited to apply for the Cell Therapy Data Engineer position at Takeda Pharmaceutical. With a Master of Science in Information Systems from Northeastern University and a background in Computer Engineering, I have developed a robust foundation in data engineering and analytics. My two years of experience as a Python developer have sharpened my skills in designing, developing, and deploying scalable data solutions.

In my previous roles, I have successfully developed a Flask application for ML-based prostate cancer diagnosis as a Graduate Research Assistant at Northeastern University, enhancing decision-making precision by 20%. At Kiddo.Health, my work as a Data Science Intern involved designing and implementing Streamlit dashboards, and optimizing data pipelines usi